<a href="https://colab.research.google.com/github/almond5/CAP5510_Datasets/blob/main/qna_method/qna_biogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Packages***

In [ ]:
%pip install transformers
%pip install sacremoses
%pip install torch
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## ***BIOGPT***

In [ ]:
import torch
from transformers import AutoTokenizer, BioGptForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=3)
ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### ***100 Samples, 3 Epochs***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(100))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 60
val_size = 20
test_size = 20

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-81cf165417f8>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.713667
2,No log,1.703198
3,No log,1.925385


Test set evaluation: {'eval_loss': 1.1954342126846313, 'eval_runtime': 2.0257, 'eval_samples_per_second': 9.873, 'eval_steps_per_second': 9.873, 'epoch': 3.0}


In [ ]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")


0
Question: Cardiovascular risk in a rural adult West African population: is resting heart rate also relevant?
Context: {'contexts': ['Elevated resting heart rate (RHR) is a neglected marker in cardiovascular risk factor studies of sub-Saharan African populations. This study aimed to determine the prevalence of elevated RHR and other risk factors for cardiovascular disease (CVD) and to investigate any associations between RHR and these risk factors in a rural population in Ghana.', 'Cross-sectional analysis.', 'A total of 574 adults aged between 18-65 years were randomly sampled from a population register. Data collected included those on sociodemographic variables and anthropometric, blood pressure (BP), and RHR measurements. Within-person variability in RHR was calculated using data from repeat measurements taken 2 weeks apart.', 'Of study participants, 36% were male. Prevalence of casual high BP was 19%. In the population, 10% were current cigarette smokers and habitual alcohol use 

### ***300 samples; 5 epochs***

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-6c1aedc3bee5>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.064112
2,No log,0.600463
3,1.318400,0.618196
4,1.318400,0.419320
5,0.583200,0.243579


Test set evaluation: {'eval_loss': 0.909318745136261, 'eval_runtime': 5.8255, 'eval_samples_per_second': 8.583, 'eval_steps_per_second': 8.583, 'epoch': 5.0}


In [ ]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")


0
Question: Fragility of the esophageal mucosa: a pathognomonic endoscopic sign of primary eosinophilic esophagitis?
Context: {'contexts': ['Primary eosinophilic esophagitis, a chronic inflammatory disorder of the esophagus, evokes recurrent dysphagia. Endoscopy is often unremarkable, and no consensus exists regarding management of resultant dysphagia. The response of a series of patients with primary eosinophilic esophagitis to dilation is reported together with a description of a possibly pathognomonic sign: fragile esophageal mucosa, for which the term "crêpe-paper" mucosa is introduced.', 'Five men underwent endoscopy because of dysphagia confirmed (clinically, endoscopically, and histologically) to be caused by primary eosinophilic esophagitis and were treated by bouginage.', 'All patients had extremely fragile, inelastic, and delicate mucosa, which tore easily even with minor trauma. After the procedure, patients remained asymptomatic for 3 to 24 months.'], 'labels': ['BACKGROUND